# Prepare C Space

Read BasicTable Excel File

In [221]:
import xlrd 
loc = 'C:\\Users\\Parveen\\Documents\\SNAPROJECT\\Project\\BasicTable.xls'

wb = xlrd.open_workbook(loc) 
sheet = wb.sheet_by_index(0) 
  
# For row 0 and column 0 
#print(sheet.cell_value(0, 0) )

Identify number of rows

In [222]:
NumberOfRows = sheet.nrows
print('Number of Rows are : ' + str(NumberOfRows))

Number of Rows are : 14960


Look for unique bus numbers

In [223]:
BusStops = []
for i in range(NumberOfRows-1):
    value = sheet.cell_value(i+1,1)
    if value not in BusStops:
        BusStops.append(value)
#print(BusStops)
print(len(BusStops))

1860


Collect all bus route numbers in which Unique bus stop appears

In [224]:
BusStopsDict = {}
for i in range(len(BusStops)):
    temp = []
    #print (BusStops[i])
    for j in range(NumberOfRows-1):
        value = sheet.cell_value(j+1,1)
        if value == BusStops[i]:
            #print(j+1+1,sheet.cell_value(j+1,0))
            temp.append(sheet.cell_value(j+1,0))
    BusStopsDict[BusStops[i]]=temp

Collect all bus stops in which Unique bus route appears

In [225]:
BusRoutesDict = {}
temp = []
for i in range(NumberOfRows-1):
    value1 = sheet.cell_value(i+1,0)
    #print(i,value1,sheet.cell_value(i+1,1))
    if value1 in BusRoutesDict:
        #print("Yes")
        temp = BusRoutesDict[value1] 
        temp.append(sheet.cell_value(i+1,1))
        BusRoutesDict[value1] = temp
    else:
        #print("No")
        BusRoutesDict[value1] = [sheet.cell_value(i+1,1)]    

Prepare an excel file where adjaceny matrix for C space will be stored

In [226]:
import xlsxwriter 
workbook = xlsxwriter.Workbook('C:\\Users\\Parveen\\Documents\\SNAPROJECT\\Project\\CSpaceTable.xlsx')
worksheet = workbook.add_worksheet() 

Just Prepare Header Names for top row and left column

In [227]:
i = 0
for key in BusRoutesDict.keys():
    worksheet.write(0,i+1,key) 
    worksheet.write(i+1,0,key)
    i += 1

Prepare a Counter List counter_list that will hold counts of overlaping bus stops for each routes

In [228]:
counter_list = []
for row,row_stop_list in BusRoutesDict.items():
    row_stops_set = set(row_stop_list)
    for col,col_stop_list in BusRoutesDict.items():
        col_stops_set = set(col_stop_list)
        temp = len(col_stops_set.intersection(row_stops_set))
        counter_list.append(temp)

In [229]:
import numpy as np
CommonBusStopsCountList = np.array((counter_list))
CommonBusStopsCountArray = CommonBusStopsCountList.reshape((737,737))
CommonBusStopsCountArray.shape

(737, 737)

In [230]:
import pandas as pd
df = pd.read_excel('C:\\Users\\Parveen\\Documents\\SNAPROJECT\\Project\\CSpaceTable.xlsx')
df2 = pd.DataFrame(B)
df2.columns = df.columns[1:]
df2.insert(0, "", df.columns[1:]) 

df2.to_csv('C:\\Users\\Parveen\\Documents\\SNAPROJECT\\Project\\C_base_table.csv',index = False)

RowsCols = CommonBusStopsCountArray.shape

B_diag_zero = np.multiply(B,1-np.eye(RowsCols[0]))
df3 = pd.DataFrame(B_diag_zero)
df3.columns = df.columns[1:]
df3.insert(0, "", df.columns[1:]) 

df3.to_csv('C:\\Users\\Parveen\\Documents\\SNAPROJECT\\Project\\C_base_table_diag_zero.csv',index = False)

import xlrd 
loc = 'C:\\Users\\Parveen\\Documents\\SNAPROJECT\\Project\\CSpaceTable.xlsx'

wb = xlrd.open_workbook(loc) 
sheet = wb.sheet_by_index(0)

Prepare Different C spaces like C10,C15,C20,..C50

In [231]:
Threshold = 20
CSpace = B_diag_zero > Threshold

In [232]:
OnesArray = np.ones((RowsCols[0],RowsCols[0]), dtype=int)
CSpace = np.multiply(OnesArray,CSpace)

In [233]:
df4 = pd.read_excel('C:\\Users\\Parveen\\Documents\\SNAPROJECT\\Project\\CSpaceTable.xlsx')
df4 = pd.DataFrame(CSpace)
df4.columns = df.columns[1:]
df4.insert(0, "", df.columns[1:]) 
filename = 'C:\\Users\\Parveen\\Documents\\SNAPROJECT\\Project\\CSpace' + str(Threshold) + '.csv'
df4.to_csv(filename,index = False)